In [4]:
%pip install pulp

In [5]:
from pulp import *
# Definimos el problema
# Jobshop para 7 máquina y 5 trabajos
J=['J1','J2','J3','J4','J5']
M=['M1','M2','M3','M4','M5','M6','M7']
O=[['M1','M2','M4','M5','M6','M7'],
   ['M6','M3','M5','M7','M2','M1'],
   ['M3','M7','M6','M1','M5','M4'],
   ['M5','M2','M7','M4','M1','M6'],
   ['M7','M1','M5','M3','M4','M2']]
# Tiempos de procesamiento [horas]
p=[[12,8,0,18,7,12,12],
   [16,11,6,0,14,9,6],
   [3,0,12,14,8,14,14],
   [15,13,0,8,9,15,16],
   [10,15,19,6,13,0,11]]
# due dates [horas]
d=[[54],[78],[54],[85],[90]]
# Creación de diccionarios
d=makeDict([J],d) 
p=makeDict([J,M],p)
O=makeDict([J,range(6)],O)

In [6]:
# Minimizar las desviaciones respecto a la fecha de entrega, 
# suponga el costo de anticipación es de 2$/hora, y de entrega tardía de 5$/hora
problema=LpProblem('Jobshop',LpMinimize)
X=LpVariable.dicts('X',((j,i) for j in J for i in M ),lowBound=0,cat='Continuos')
Y=LpVariable.dicts('Y',((v,q,i) for v in J for q in J for i in M),lowBound=0, upBound=1,cat='Binary')
E=LpVariable.dicts('E',(j for j in J),lowBound=0,cat='Continuos')
T=LpVariable.dicts('T',(j for j in J),lowBound=0,cat='Continuos')

for j in J:
    h=O[j][len(O[j])-1]
    problema+=X[j,h]+p[j][h]-T[j]+E[j]==d[j]

for j in J:
  for r in range(len(O[j])-1):
      h=O[j][r]
      h1=O[j][r+1]
      problema+=X[j,h]+p[j][h]<=X[j,h1]

MM=1000
for i in M:
  for v in J:
    for q in J:
      if(q!=v):
          problema+=X[v,i]+p[v][i]<=X[q,i]+MM*(1-Y[v,q,i])
          problema+=X[q,i]+p[q][i]<=X[v,i]+MM*(Y[v,q,i])

problema+= 2*lpSum(E[j] for j in J)+5*lpSum(T[j] for j in J)

problema.writeLP('c.lp')

problema.solve()
print(LpStatus[problema.status])
print("Function = ", problema.objective, "result = ", value(problema.objective))
for j in J:
    for r in O[j]:
        i=O[j][r]
        print(j,".",i,".", value(X[j,i]))

Optimal
Function =  2*E_J1 + 2*E_J2 + 2*E_J3 + 2*E_J4 + 2*E_J5 + 5*T_J1 + 5*T_J2 + 5*T_J3 + 5*T_J4 + 5*T_J5 result =  290.0
J1 . M1 . 0.0
J1 . M2 . 22.0
J1 . M4 . 30.0
J1 . M5 . 53.0
J1 . M6 . 60.0
J1 . M7 . 72.0
J2 . M6 . 0.0
J2 . M3 . 12.0
J2 . M5 . 18.0
J2 . M7 . 42.0
J2 . M2 . 48.0
J2 . M1 . 71.0
J3 . M3 . 0.0
J3 . M7 . 12.0
J3 . M6 . 26.0
J3 . M1 . 40.0
J3 . M5 . 45.0
J3 . M4 . 56.0
J4 . M5 . 0.0
J4 . M2 . 9.0
J4 . M7 . 26.0
J4 . M4 . 48.0
J4 . M1 . 56.0
J4 . M6 . 72.0
J5 . M7 . 0.0
J5 . M1 . 22.0
J5 . M5 . 32.0
J5 . M3 . 51.0
J5 . M4 . 70.0
J5 . M2 . 76.0
